# Data frame optimization

The heaviest task we want to perform on the input data frame consists on appending audio snippets.
This involves opening a `.wav` file for each row.

Some of these rows points to the same `.wav` file, so we'll make sure the file is opened only once.

## Input dataframe

In [1]:
from pandas import read_csv

In [2]:
df = read_csv("tests/test_input.csv")
df

,start_time,end_time,participant,utterance,key,language,uid
0,629.960,630.510,A,aha,/german1/5298,german,german-059-255-629960
1,398.870,399.330,A,aha,/german1/5298,german,german-059-151-398870
2,2009.100,2009.500,tx@ADUSBS,aoq,/sambas1/SBS-20111031,sambas,sambas-24-0883-2009100
3,1782.890,1783.400,tx@JEPSBS,aoq,/sambas1/SBS-20111031,sambas,sambas-24-0764-1782890
4,341.410,341.830,B,mhm,/german1/4123,german,german-008-097-341410
5,622.020,622.370,A,ja,/german1/4123,german,german-008-223-622020
6,220.343,220.682,f37ln,sí,/catalan1/ca_f37s_f38s_und,catalan,catalan-12-091-220343
7,266.974,267.346,f37ln,sí,/catalan1/ca_f37s_f38s_und,catalan,catalan-12-108-266974
8,145.130,145.820,tx@39,yeah,/arapaho1/25b,arapaho,arapaho-22-076-145130
9,417.900,418.310,tx@5,yeah,/arapaho1/25b,arapaho,arapaho-22-206-417900


Please note the times are in seconds.

## Auxiliary functions:
This adapter will help us converting our syntax (using keys) into librosa's syntax (using filenames).

In [3]:
from corpusparser.auxs import filename_from_key
filename_from_key("/catalan1/ca_f02a_m05a_und")

'data/catalan1/ca_f02a_m05a_und.wav'

## Extract audio features

In [4]:
from corpusparser.parsers import *

### Example of usage

### Extract all audio

In [5]:
audio_from_key("/catalan1/ca_f02a_m05a_und")

array([-0.00175476, -0.00236511, -0.00218201, ...,  0.        ,
        0.        ,  0.        ], dtype=float32)

### Extract sample rate

In [6]:
samplerate_from_key("/catalan1/ca_f02a_m05a_und")

16000

### Append all sample rates

In [7]:
df['rate'] = samplerate_from_keys(df['key'])
df.head(3)


,start_time,end_time,participant,utterance,key,language,uid,rate
0,629.96,630.51,A,aha,/german1/5298,german,german-059-255-629960,8000
1,398.87,399.33,A,aha,/german1/5298,german,german-059-151-398870,8000
2,2009.10,2009.50,tx@ADUSBS,aoq,/sambas1/SBS-20111031,sambas,sambas-24-0883-2009100,96000


### Extract an audio snippet

In [8]:
key = "/catalan1/ca_f02a_m05a_und"
df[df["key"] == key].reset_index()

,index,start_time,end_time,participant,utterance,key,language,uid,rate
0,10,318.486,318.890,f02lp,sí,/catalan1/ca_f02a_m05a_und,catalan,catalan-08-222-318486,16000
1,11,84.358,84.546,m05lp,sí,/catalan1/ca_f02a_m05a_und,catalan,catalan-08-061-84358,16000


In [9]:
snippet = subset_audio_from_key(df, key, row=0)
snippet

array([-0.00238037, -0.01153564, -0.00408936, ..., -0.01112366,
       -0.01062012, -0.01023865], dtype=float32)

### Append all audio snippets to dataframe

In [10]:
size = len(df)
snippets = size * [None] # Pre-allocate an empty list

counter = 0
keys = df['key'].unique()
for key in keys:
    # Open the audio file only once per file (as opposed to once per row)
    audio = audio_from_key(key)

    # Extract and append the relevant audio snippet
    aux = df[df['key'] == key]
    for i, row in aux.iterrows():
        snippets[counter] = subset_audio(audio, row['start_time'], row['end_time'], row['rate'])
        counter += 1
    
df.insert(len(df.columns), "audio", snippets)

In [11]:

df

,start_time,end_time,participant,utterance,key,language,uid,rate,audio
0,629.960,630.510,A,aha,/german1/5298,german,german-059-255-629960,8000,"[-0.00012207031, -0.00061035156, -0.0008544922..."
1,398.870,399.330,A,aha,/german1/5298,german,german-059-151-398870,8000,"[-0.0009765625, -0.0008544922, -0.0010986328, ..."
2,2009.100,2009.500,tx@ADUSBS,aoq,/sambas1/SBS-20111031,sambas,sambas-24-0883-2009100,96000,"[0.008453369, 0.008483887, 0.007232666, 0.0072..."
3,1782.890,1783.400,tx@JEPSBS,aoq,/sambas1/SBS-20111031,sambas,sambas-24-0764-1782890,96000,"[-0.012969971, -0.012939453, -0.011016846, -0...."
4,341.410,341.830,B,mhm,/german1/4123,german,german-008-097-341410,8000,"[-0.0020141602, -0.0015869141, -0.0014648438, ..."
5,622.020,622.370,A,ja,/german1/4123,german,german-008-223-622020,8000,"[0.00024414062, 0.0, 0.0, -0.00024414062, -0.0..."
6,220.343,220.682,f37ln,sí,/catalan1/ca_f37s_f38s_und,catalan,catalan-12-091-220343,16000,"[-0.0016174316, -0.0015258789, -0.0014343262, ..."
7,266.974,267.346,f37ln,sí,/catalan1/ca_f37s_f38s_und,catalan,catalan-12-108-266974,16000,"[0.00019836426, -0.00044250488, -0.0014038086,..."
8,145.130,145.820,tx@39,yeah,/arapaho1/25b,arapaho,arapaho-22-076-145130,44100,"[0.006210327, 0.0040740967, 0.0043640137, 0.00..."
9,417.900,418.310,tx@5,yeah,/arapaho1/25b,arapaho,arapaho-22-206-417900,44100,"[0.023712158, 0.022613525, 0.025222778, 0.0259..."


In [12]:
from IPython.display import Audio
index = 0
#Audio(data = df["audio"][index], rate = df["rate"][index])

## (Optional) Listen to the snippets

### Auxiliary functions

In [13]:
from corpusparser.listeners import listen_audio_from_key

key = "/catalan1/ca_f02a_m05a_und"
#listen_audio_from_key(df, key = key, row = 0)